# train / dev / eval protocol 만들기

In [ ]:
import csv
import os
from collections import defaultdict

csv_path = "/home/woongjae/ADD_Dataset_SpeechOnly/Dataset/20260204/metadata.csv"   # 네 CSV 파일 경로
output_dir = "/home/woongjae/ADD_Dataset_SpeechOnly/protocols"
os.makedirs(output_dir, exist_ok=True)

files = {
    "train": open(os.path.join(output_dir, "speechonly_train.txt"), "w"),
    "dev":   open(os.path.join(output_dir, "speechonly_dev.txt"), "w"),
    "eval":  open(os.path.join(output_dir, "speechonly_eval.txt"), "w"),
}

with open(csv_path, newline="", encoding="utf-8") as f:
    reader = csv.DictReader(f)
    for row in reader:
        subset = row["subset"].strip()
        if subset not in files:
            continue

        filename = row["current_filename"].strip()
        label = row["label"].strip()

        # 핵심 포맷
        files[subset].write(f"{filename} {subset} {label}\n")

for f in files.values():
    f.close()

print("✅ Protocol files generated with format: file_name subset label")

✅ Protocol files generated with label mapping (real/fake → bonafide/spoof)


In [7]:
protocol_dir = "/home/woongjae/ADD_Dataset_SpeechOnly/protocols"  # 프로토콜 저장한 디렉토리
files = [
    "20260204_speechonly_train.txt",
    "20260204_speechonly_dev.txt",
    "20260204_speechonly_eval.txt",
]

total = 0
counts = {}

for fname in files:
    path = os.path.join(protocol_dir, fname)
    if not os.path.exists(path):
        print(f"❌ {fname} not found")
        counts[fname] = 0
        continue

    with open(path, "r", encoding="utf-8") as f:
        count = sum(1 for _ in f)

    counts[fname] = count
    total += count

# 출력
for fname, cnt in counts.items():
    print(f"📄 {fname}: {cnt} samples")

print("-" * 40)
print(f"📊 TOTAL samples (train+dev+eval): {total}")

📄 20260204_speechonly_train.txt: 223569 samples
📄 20260204_speechonly_dev.txt: 81324 samples
📄 20260204_speechonly_eval.txt: 158092 samples
----------------------------------------
📊 TOTAL samples (train+dev+eval): 462985


### csv 파일에서 label열 real/fake -> bonafide/spoof

In [8]:
import csv
import os

input_csv = "/home/woongjae/ADD_Dataset_SpeechOnly/Dataset/20260204/metadata.csv"
output_csv = "/home/woongjae/ADD_Dataset_SpeechOnly/Dataset/20260204/metadata1.csv"

label_map = {
    "real": "bonafide",
    "fake": "spoof",
}

with open(input_csv, newline="", encoding="utf-8") as fin, \
     open(output_csv, "w", newline="", encoding="utf-8") as fout:

    reader = csv.DictReader(fin)
    fieldnames = reader.fieldnames
    writer = csv.DictWriter(fout, fieldnames=fieldnames)

    writer.writeheader()

    for row in reader:
        raw_label = row["label"].strip().lower()
        if raw_label not in label_map:
            raise ValueError(f"Unknown label: {raw_label}")

        row["label"] = label_map[raw_label]
        writer.writerow(row)

print(f"✅ Metadata label converted and saved to: {output_csv}")


✅ Metadata label converted and saved to: /home/woongjae/ADD_Dataset_SpeechOnly/Dataset/20260204/metadata1.csv
